In [8]:
import json

In [9]:
vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(r"vocabs\en\en-len5-hard.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

with open(r"vocabs\en\en-len5-medium.json") as file:
    json_data = json.load(file)
    for d in json_data:
        exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), alphabets)
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

26 {'a', 'k', 'f', 'h', 'b', 'd', 'v', 'm', 's', 'i', 'e', 'g', 'n', 'l', 'c', 'q', 'j', 'o', 't', 'x', 'y', 'z', 'u', 'r', 'w', 'p'}
len(vocabs_pool)=1500
len(exclusive_pool)=500


In [10]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'abk', 'amc', 'vbc', 'avc', 'abl', 'abp', 'axc', 'afc', 'rbc', 'azc', 'abv', 'asc', 'aby', 'abt', 'ebc', 'tbc', 'aec', 'abb', 'aic', 'xbc', 'qbc', 'abr', 'abw', 'abu', 'kbc', 'pbc', 'agc', 'abs', 'arc', 'aqc', 'abi', 'abj', 'acc', 'zbc', 'aoc', 'cbc', 'abm', 'abo', 'bbc', 'aac', 'aba', 'adc', 'abz', 'sbc', 'akc', 'obc', 'alc', 'apc', 'abq', 'dbc', 'lbc', 'fbc', 'abf', 'abx', 'abg', 'ybc', 'awc', 'abh', 'hbc', 'abd', 'jbc', 'atc', 'auc', 'anc', 'abe', 'nbc', 'wbc', 'ibc', 'mbc', 'ahc', 'ayc', 'abn', 'ajc', 'gbc', 'ubc'}


In [11]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [12]:
def distance(from_: str, to: str, max_distance=None) -> tuple[int, list[str]]:
    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [13]:
print(distance("admin", "admit"))

(1, ['admin', 'admit'])


In [14]:
max_distance = 5
count = 0
count_exclusive = 0
starter = set()
involved = set()
for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        diff = character_difference(v1, v2)
        if dist[0] > diff:
            count += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                count_exclusive += 1

    if i % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i}/{num_vocabs}')

total_combinations = num_vocabs * (num_vocabs - 1)

pair_rate = count / total_combinations
print(f"pair rate: {count}/{total_combinations} = {pair_rate:.2%}")

exclusive_rate = count_exclusive / total_combinations
print(
    f"exclusive pair rate: {count_exclusive}/{total_combinations} = {exclusive_rate:.2%}"
)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

checking "lynch"| 0/1500
checking "clone"| 150/1500
checking "today"| 300/1500
checking "first"| 450/1500
checking "punch"| 600/1500
checking "singh"| 750/1500
checking "scott"| 900/1500
checking "opera"| 1050/1500
checking "allen"| 1200/1500
checking "armed"| 1350/1500
pair rate: 13022/2248500 = 0.58%
exclusive pair rate: 11432/2248500 = 0.51%
starter: 560/1500 = 37.33%
participation: 588/1500 = 39.20%


In [15]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)